# 5.4.2 - GroupBy in Spark

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate() 

25/03/27 21:53:01 WARN Utils: Your hostname, DESKTOP-BHSOG44 resolves to a loopback address: 127.0.1.1; using 192.168.147.61 instead (on interface eth0)
25/03/27 21:53:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/27 21:53:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/27 21:53:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.createOrReplaceTempView('green')

df_green：这是一个 PySpark DataFrame，代表一个分布式数据集。

.createOrReplaceTempView('green')：这个方法会在 Spark SQL 中创建一个临时视图（或替换已有的同名视图），视图名称是 'green'。

临时视图（Temporary View）：仅在当前 SparkSession 中有效，会话结束后自动消失，不会持久化到元数据库（如 Hive Metastore）。

createOrReplace：如果 'green' 视图已存在，则替换它；否则创建新视图。

In [8]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [9]:
df_green_revenue.show()

[Stage 6:>                                                          (0 + 4) / 4]

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999997|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000001|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [10]:
df_green_revenue \
     .repartition(20) \ #make them a little bit bigger
     .write.parquet('data/report/revenue/green')

repartition(N)：强制调整分区数（触发 Shuffle），适合解决数据倾斜或控制文件数量。

增大分区数：当数据倾斜、并行度不足或需要更多输出文件时。

减小分区数：当分区过多（小文件问题）时，优先用 coalesce。

In [ ]:
df_green_revenue.write.parquet('data/report/revenue/green', mode='overwrite') #cover the previous

In [11]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [12]:
df_yellow.createOrReplaceTempView('yellow')

In [15]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [16]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   3|              25.0|             1|
|2020-01-01 00:00:00|   4|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7| 455.1700000000001|            38|
|2020-01-01 00:00:00|  10|             42.41|             2|
|2020-01-01 00:00:00|  12|             107.0|             6|
|2020-01-01 00:00:00|  13|            1214.8|            56|
|2020-01-01 00:00:00|  14|               8.8|             1|
|2020-01-01 00:00:00|  15|             34.09|             1|
|2020-01-01 00:00:00|  17|220.20999999999998|             8|
|2020-01-01 00:00:00|  18|               5.8|             1|
|2020-01-01 00:00:00|  24| 754.9500000000003|            45|
|2020-01-01 00:00:00|  25|            324.35|            16|
|2020-01-01 00:00:00|  32|              18.0|             1|
|2020-01-01 00:00:00|  3

In [19]:
df_yellow_revenue \
     .repartition(20) \
     .write.parquet('data/report/revenue/yellow', mode='overwrite')

# 5.4.3 - Joins in Spark

When 2 datasets are too large, we can use parquet instead of original df_green_revenue

In [29]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [30]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [31]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [32]:
df_join.show()

[Stage 66:===========================================>              (3 + 1) / 4]

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|  13|              null|                null|            1214.8|                   56|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  36|295.34000000000003|                  11|            109.17|                    3|
|2020-01-01 00:00:00|  45|              null|                null| 732.4800000000002|                   42|
|2020-01-01 00:00:00|  59|50.900000000000006|                   3|              null|                 null|
|2020-01-01 00:00:00|  61|  

In [33]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [34]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

When one dataset is large and another is small

In [35]:
df_zones = spark.read.parquet('zones/')

In [36]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [37]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [38]:
df_result.drop('LocationID').show()

+-------------------+----+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|  13|              null|                null|            1214.8|                   56|Manhattan|   Battery Park City| Yellow Zone|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|    Bronx|           Bronxdale|   Boro Zone|
|2020-01-01 00:00:00|  36|295.34000000000003|                  11|            109.17|   

In [39]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')